In [62]:
# 1. import libraries
import requests # to download html code
from bs4 import BeautifulSoup # to navigate through the html code
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans



import config
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials

#Initialize SpotiPy with user credentias
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))

hot100_df = pd.read_csv("artistsong.csv")
clustered_df = pd.read_csv("clustered.csv")

In [52]:
hot100_df

,artist,song
0,miley cyrus,flowers
1,sza,kill bill
2,morgan wallen,last night
3,sam smith & kim petras,unholy
4,"metro boomin, the weeknd & 21 savage",creepin'
...,...,...
95,yandel & feid,yandel 150
96,mac demarco,heart to heart
97,jordan davis,next thing you know
98,p!nk,never gonna not dance again


In [53]:
import pyinputplus as pyip   #to insert predefined user input functions
import random

user_song = str(input("Choose your song: "))    
rec_song = hot100_df["song"].sample().to_string(index=False)     #randomly reccomend another song with sample, .to_string avoit to put the index too
rec_artist = hot100_df.loc[hot100_df["song"] == rec_song, "artist"].to_string(index=False)   #associate the artist to the random reccomended song

x_artist = hot100_df.loc[hot100_df['song'].map(str.lower)  == user_song.lower(), 'artist'] # artist corresponding to song x
artist_question = pyip.inputYesNo('Did you mean ' + user_song.lower().title() + ' by ' + x_artist.to_string(index=False).title() + '?')  #this asks if the user mean an identical song but not in the list, if neither Y or N are given, it asks again

if artist_question == "yes":
    if user_song.lower() in hot100_df.values and user_song.lower() != rec_song:         #.lower() exclude typos from capital/lower letter "and user_song.lower() != rec_song" exclude to randomize including the same song
        print(f"The song you have chosen is " + user_song.title() + ". Why don't you try to listen to " + rec_song.title() + " by " + rec_artist.title())     #.title() gives results with capital first letter
    else:
        print("Your song is not in Billboard 100 Hot")
else:
    print("Your song is not in Billboard 100 Hot")

Choose your song: f
Did you mean F by Series([], )?f
'f' is not a valid yes/no response.
Did you mean F by Series([], )?n
Your song is not in Billboard 100 Hot


In [54]:
#User function again
x = str(input("Choose your song: "))
results = sp.search(q=x, limit=3,market="GB")
results

Choose your song: One Love


{'tracks': {'href': 'https://api.spotify.com/v1/search?query=One+Love&type=track&market=GB&offset=0&limit=3',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2QsynagSdAqZj3U9HgDzjD'},
       'href': 'https://api.spotify.com/v1/artists/2QsynagSdAqZj3U9HgDzjD',
       'id': '2QsynagSdAqZj3U9HgDzjD',
       'name': 'Bob Marley & The Wailers',
       'type': 'artist',
       'uri': 'spotify:artist:2QsynagSdAqZj3U9HgDzjD'}],
     'external_urls': {'spotify': 'https://open.spotify.com/album/2mBbV0Ad6B4ydHMZlzAY7S'},
     'href': 'https://api.spotify.com/v1/albums/2mBbV0Ad6B4ydHMZlzAY7S',
     'id': '2mBbV0Ad6B4ydHMZlzAY7S',
     'images': [{'height': 640,
       'url': 'https://i.scdn.co/image/ab67616d0000b273811bfdc4f55b3a7484524aaf',
       'width': 640},
      {'height': 300,
       'url': 'https://i.scdn.co/image/ab67616d00001e02811bfdc4f55b3a7484524aaf',
       'width': 300},
      {'height': 64,
       'url':

In [55]:
#From the song to the track id
track_id=track_id=results["tracks"]["items"][0]["id"]
track_id

'6SR0dDuXH2vtgBfMLrivKV'

In [56]:
#Just try the display of the song
from IPython.display import IFrame

#track_id = "1rfORa9iYmocEsnnZGMVC4"
#track_id= 'spotify:track:3hgl7EQwTutSm6PESsB7gZ'
IFrame(src="https://open.spotify.com/embed/track/"+track_id,
       width="320",
       height="80",
       frameborder="0",
       allowtransparency="true",
       allow="encrypted-media",
      )

In [57]:
#audio feature of the chosen song
x_af = sp.audio_features(track_id)
X_af = pd.DataFrame(x_af)
X_af

X_af_num = X_af.drop(['key', 'mode', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature'], axis=1)
# dropping some unwanted columns
X_af_num

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.67,0.516,-9.598,0.366,0.0673,0,0.0674,0.939,154.186


In [64]:
#Scale the chosen song to the scaled used for our spotify dataset
hot_df = pd.read_csv("hot100_billboard.csv")
df_for_scaling = hot_df.drop(["id", "duration_ms"], axis=1) #spotify data without duration and id

scaler = StandardScaler()
scaler_fitted = scaler.fit(df_for_scaling) #scaling for fitting of the new song
X_scaled = scaler_fitted.transform(df_for_scaling)
X_scaled_df = pd.DataFrame(X_scaled, columns = df_for_scaling.columns)

X_af_num_scaled = scaler_fitted.transform(X_af_num)
X_af_num_scaled


array([[ 0.38583971, -0.48043877, -0.30050489,  4.62491319, -0.69633509,
        -0.36282265, -0.75633579,  1.49912139,  1.3301458 ]])

In [65]:
#Predicting the cluster of the chosen song

kmeans = KMeans(n_clusters=7, random_state=1234)
kmeans.fit(X_scaled_df)
input_song_cluster = kmeans.predict(X_af_num_scaled)

/Users/danilogiarlini/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
